Notebook utilizado para reordenar as histórias de um épico com base no ordenamento dos status das histórias do projeto.

Pode se utilizar um ordenamento próprio caso deseje substituir o ordenamento padrão do projeto.

Reordenador de tarefas do épico.

In [ ]:
%pip install python-taiga

In [1]:
from taiga import TaigaAPI
import getpass
from taiga.models.models import UserStory
from taiga.requestmaker import RequestMaker
import time

In [2]:
hostname = input("Informe o hostname: ")
if(not hostname or not hostname.strip()):
    raise Exception("Hostname não informado!")

token = getpass.getpass("Informe o token: ")
if(not token or not token.strip()):
    raise Exception("Token não informado!")


api = TaigaAPI(host=hostname,token=token)




# matricula = input("Informe a matrícula: ")
# senha = getpass.getpass("Informe a senha: ")

## Autenticação via LDAP no taiga
# api = TaigaAPI(
#     host= hostname,
#     auth_type='ldap'
# )
# api.auth(
#     username=matricula,
#     password=senha
# )


In [ ]:
api.me()

Coletando o nome do projeto

In [6]:
nome_projeto = input("Informe o nome do projeto: ")
if(not nome_projeto or not nome_projeto.strip()):
    raise Exception("Nome do projeto não informado!")

Recuperando o projeto.

In [ ]:
lista_projetos = api.projects.list(q=nome_projeto)
for projetoItem in lista_projetos:
    print(f"\tProjeto: {projetoItem}")
    print(f"\t\tid: {projetoItem.id}")

print("\n")
projeto = None
if(lista_projetos):
    for projetoItem in lista_projetos:
        if(nome_projeto == projetoItem.name):
            projeto = projetoItem
            print(f"Projeto encontrado {projeto} - {projetoItem.id}")
            break
    if(not projeto):
        raise Exception("Projeto não encontrado! (1)")
else:
    raise Exception("Projeto não encontrado! (2)")

Coletando o nome do épico de entrada.

In [8]:
epico_entrada = input("Informe o nome do épico de entrada: ")
if(not epico_entrada or not epico_entrada.strip()):
    raise Exception("Épico de entrada não informado!")

Recuperando o épico de entrada.

In [ ]:
lista_epicos = api.epics.list(project=projeto.id)
for epic_item in lista_epicos:
    print (f"\t\tEpico: {epic_item}")

In [10]:
def coletar_epico_por_nome_em_lista(nome_epico="", lista_epicos=[]):
    epico_obj = None
    for epic_item in lista_epicos:
        if(nome_epico == epic_item.subject):
            print (f"\t\tEpico: {epic_item}")
            print (f"\t\t\tRef: {epic_item.ref}")
            print (f"\t\t\tSubject: {epic_item.subject}")
            epico_obj = projeto.get_epic_by_ref(epic_item.ref)
            print (f"\t\t\tDescription: {epico_obj.description}")
            print (f"\t\t\tAttributes: {epico_obj.get_attributes()}")
            print (f"\t\t\tto_dict(): {epico_obj.to_dict()}")
    return epico_obj

Filtrando o épico de entrada pela lista.

In [ ]:
epico_entrada_obj = coletar_epico_por_nome_em_lista(nome_epico=epico_entrada, lista_epicos=lista_epicos)
if(not epico_entrada_obj):
    raise Exception("Épico não encontrado!")

Listando as histórias do épico.

In [ ]:
request_maker_obj = RequestMaker('/api/v1/', hostname, token)

# Consultando a lista de histórias
mapa_get_historias = {}
mapa_get_historias['epic'] = epico_entrada_obj.id
mapa_get_historias['include_tasks'] = 'true'
mapa_get_historias['order_by'] = 'epic_order'

print(mapa_get_historias)
retorno = request_maker_obj.get(f'userstories', query=mapa_get_historias)

lista_historias = retorno.json()
type(lista_historias)


Coletando os status das histórias do projeto.

In [ ]:
lista_status_user_histories = projeto.list_user_story_statuses()
print(lista_status_user_histories)

for status_historia in lista_status_user_histories:
    print(f"UserStoryStatus: projeto: {status_historia.project}, - name: {status_historia.name}, - order: {status_historia.order}")


Criando o mapa de ordenação dos status das histórias.

In [ ]:
mapa_status_ordem = {}
for status_historia in lista_status_user_histories:
    mapa_status_ordem[status_historia.name] = status_historia.order

print(mapa_status_ordem)

def get_codigo_ordem_por_descricao_status(status='', mapa_status_ordem_param = {}):
    # print(f"status: {status} - retorno: {mapa_status_ordem_param[status]}")
    return mapa_status_ordem_param[status]

Modificando a ordem padrão dos status.

In [15]:
# mapa_status_ordem = {'New': 1, 'Ready': 2, 'In progress': 3, 'Ready for test': 4, 'Done': 5, 'Archived': 6}

Prévia de ordenação das histórias do épico.

In [ ]:
for historia in sorted(lista_historias, key = lambda item: get_codigo_ordem_por_descricao_status(status=item['status_extra_info']['name'], mapa_status_ordem_param = mapa_status_ordem)):
    print(f"História: {historia['ref']} - {historia['subject']}")

Aplicando a ordenação das histórias do épico.

In [ ]:
for idx_historia_ordenada, historia in enumerate(sorted(lista_historias, key = lambda item: get_codigo_ordem_por_descricao_status(status=item['status_extra_info']['name'], mapa_status_ordem_param = mapa_status_ordem))):

    print(f"História: {historia['ref']} - {historia['subject']}")

    # Consultando a lista de histórias
    mapa_patch_ordenamento = {}
    mapa_patch_ordenamento['order'] = lista_historias[idx_historia_ordenada]['epic_order']

    print(mapa_patch_ordenamento)
    retorno = request_maker_obj.patch(f'epics/{epico_entrada_obj.id}/related_userstories/{historia['id']}', payload=mapa_patch_ordenamento)

    time.sleep(0.5)
